# Pitch Classes as a Vector Space in ℝ¹²

We represent the 12 pitch classes (ignoring octave) as vectors in the real space:

$$
\mathbb{R}^{12}
$$

The dimensions correspond to the pitch classes:

$$
\{C, C\#, D, D\#, E, F, F\#, G, G\#, A, A\#, B\}
$$

---

### Encoding Notes as Vectors

A single musical note is represented as a **one-hot vector** in ℝ¹², meaning it has a 1 in the coordinate of its pitch class and 0 elsewhere.

Example:

$$
\mathbf{e}_C = (1,0,0,0,0,0,0,0,0,0,0,0)
$$

$$
\mathbf{e}_G = (0,0,0,0,0,0,0,1,0,0,0,0)
$$

If multiple notes sound simultaneously (a chord), we **add** their vectors:

For the C-major triad \( C, E, G \):

$$
\mathbf{v}_{Cmaj} = \mathbf{e}_C + \mathbf{e}_E + \mathbf{e}_G
$$

---

### Basis of the Pitch-Class Vector Space

The **standard basis** of this vector space is the set:

$$
\mathcal{B} = \{\mathbf{e}_C, \mathbf{e}_{C\#}, \mathbf{e}_D, \mathbf{e}_{D\#},
\mathbf{e}_E, \mathbf{e}_F, \mathbf{e}_{F\#}, \mathbf{e}_G, \mathbf{e}_{G\#},
\mathbf{e}_A, \mathbf{e}_{A\#}, \mathbf{e}_B\}
$$

Each basis vector has entries:

$$
\mathbf{e}_p(i) = 
\begin{cases}
1 & \text{if } i = p, \\
0 & \text{otherwise}
\end{cases}
$$

These vectors:

- are linearly independent,
- span all pitch-class combinations,
- and allow **chords, melodies, and harmonic constraints** to be expressed algebraically.

---

### Interpretation Summary

| Musical Object | Representation |
|----------------|----------------|
| Single note | One basis vector in ℝ¹² |
| Chord | Sum of multiple basis vectors |
| Melody | Sequence of vectors in ℝ¹² |
| Scale rules | Linear constraints on coordinates |
| Composition | Solving equations in ℝ¹² |

---

This establishes a rigorous linear-algebraic framework for music analysis, transformation, and generative composition.


In [2]:
import numpy as np
import pandas as pd

In [3]:
import sounddevice as sd

In [4]:
notes = pd.read_csv("ALA.csv")
notes.head()

,Note,Octave,Frequency(Hz),log(Frequency)
0,C3,3,130.81,7.031329
1,C#3,3,138.59,7.114679
2,D3,3,146.83,7.198003
3,D#3,3,155.56,7.281327
4,E3,3,164.81,7.364660


In [5]:
C4 = 261.63
freq_4 = np.array([C4 * 2**(k/12) for k in range(12)])  # C4..B4
T4 = np.diag(freq_4)




def play_note(freq, duration, amp=0.2, samplerate=44100):
    t = np.linspace(0, duration, int(samplerate * duration), False)
    wave = amp * np.sin(2 * np.pi * freq * t)
    sd.play(wave, samplerate)
    sd.wait()

def play_C():
    ref = 261.63
    Cfreq = np.array([ref*(2**x) for x in range(-2,3)])
    return Cfreq

def play_notes(arr, duration=0.7, amp=0.2, samplerate=44100):
    for f in arr:
        t = np.linspace(0, duration, int(samplerate*duration), False)
        wave = amp * np.sin(2 * np.pi * f * t)
        sd.play(wave, samplerate)
        sd.wait()  

def generate_transform(octave):
    factor = 2**(octave-4)
    return factor * T4

def play_chord(freqs, duration=2, amp=0.2, samplerate=44100):
    t = np.linspace(0, duration, int(samplerate * duration), False)
    wave = np.zeros_like(t)
    for f in freqs:
        wave += np.sin(2 * np.pi * f * t)
    wave = amp * (wave / len(freqs))
    sd.play(wave, samplerate)
    sd.wait()
    
def sargam_vector(f_sa):
    semitones = np.array([0, 2, 4, 5, 7, 9, 11, 12], dtype=float)
    return f_sa * (2 ** (semitones / 12))

def play_sargam(note):
    freq = notes.loc[notes["Note"] == note, "Frequency(Hz)"].iloc[0]
    vector = sargam_vector(freq)
    play_notes(vector, 1.5)
    play_notes(vector[::-1],1.5)


In [113]:
play_sargam("C3")

In [7]:
def play_piano_like_tone(freq, duration=0.8, sr=44100):
    """
    Play a 'piano-like' tone at given frequency using additive overtones
    and an amplitude envelope, instead of a single sine beep.
    """
    t = np.linspace(0, duration, int(sr * duration), endpoint=False)

    # --- Harmonic structure (choose whatever sounds good) ---
    # Fundamental and first few overtones
    harmonics = np.array([1, 2, 3, 4, 5, 6, 7, 8], dtype=float)

    # Relative amplitudes (higher partials are quieter)
    amps = np.array([1.0, 0.6, 0.4, 0.25, 0.18, 0.12, 0.08, 0.05])

    # Small inharmonicity to feel more piano-like (optional)
    # Try b = 0.0001–0.0004; set to 0.0 if you want pure harmonics.
    b = 0.00025
    freqs = harmonics * freq * (1.0 + b * harmonics**2)

    # --- Additive synthesis: sum of many sines ---
    signal = np.zeros_like(t)
    for f_h, a_h in zip(freqs, amps):
        signal += a_h * np.sin(2 * np.pi * f_h * t)

    # --- Amplitude envelope: quick attack, exponential decay ---
    attack_time = 0.01  # 10 ms
    attack_samples = int(sr * attack_time)

    envelope = np.exp(-7.0 * t / duration)  # exponential decay
    envelope[:attack_samples] *= np.linspace(0, 1, attack_samples)

    signal *= envelope

    # --- Normalize to avoid clipping ---
    max_val = np.max(np.abs(signal))
    if max_val > 0:
        signal = 0.8 * signal / max_val

    sd.play(signal, sr)
    sd.wait()


In [110]:
def play_sargam_piano(f_sa):
    freqs = play(f_sa)
    for f in freqs:
        play_piano_like_tone(f)

# Shift Matrix for Pitch Classes in $ \mathbb{R}^{12} $

We model the 12 pitch classes (mod 12) as basis vectors in the vector space $ \mathbb{R}^{12} $.

We fix the ordering:

| Index | Note  |
|-------|-------|
| 0     | C     |
| 1     | C♯/D♭ |
| 2     | D     |
| 3     | D♯/E♭ |
| 4     | E     |
| 5     | F     |
| 6     | F♯/G♭ |
| 7     | G     |
| 8     | G♯/A♭ |
| 9     | A     |
| 10    | A♯/B♭ |
| 11    | B     |

Each pitch class is represented as a **one–hot basis vector** in $ \mathbb{R}^{12} $:

$$
e_0 = (1,0,0,0,0,0,0,0,0,0,0,0)^T,
\qquad
e_7 = (0,0,0,0,0,0,0,1,0,0,0,0)^T
$$

---

## 1. Semitone Shift Matrix $S$

The **semitone shift matrix** $ S \in \mathbb{R}^{12 \times 12} $ is defined by:

$$
S e_k = e_{(k+1) \bmod 12}, \qquad k = 0,1,\dots,11
$$

It shifts each pitch **up by one semitone**, wrapping after B back to C.

Matrix form:

$$
S =
\begin{bmatrix}
0 & 0 & \cdots & 0 & 1 \\
1 & 0 & \cdots & 0 & 0 \\
0 & 1 & \cdots & 0 & 0 \\
\vdots & \ddots & \ddots & \vdots & \vdots \\
0 & 0 & \cdots & 1 & 0
\end{bmatrix}
$$

---

## 2. Powers of $S$ = Transposition

The $k$-th power of $S$ transposes by $k$ semitones:

$$
S^k e_j = e_{(j+k) \bmod 12}
$$

Examples:

- $S^2 e_0 = e_2$ (C → D)  
- $S^7 e_0 = e_7$ (C → G)  
- $S^{12} = I_{12}$ (one octave return)

> **Transposition in pitch-class space = multiplication by $S^k$.**

---

## 3. Chords as Vectors

Example: C major chord (C–E–G):

- C → index 0
- E → index 4
- G → index 7

Chord vector:

$$
v_{\text{Cmaj}} = e_0 + e_4 + e_7
= (1,0,0,0,1,0,0,1,0,0,0,0)^T
$$

Transpose this chord by two semitones:

$$
S^2 v_{\text{Cmaj}} = e_2 + e_6 + e_9
$$

This is D–F♯–A (D major).

---

## 4. Group Interpretation

Pitch classes form the cyclic group:

$$
\mathbb{Z}_{12} = \{0,1,\dots,11\}
$$

Mapping:

$$
k \mapsto S^k
$$

is a **group representation**, since:

$$
S^{k_1} S^{k_2} = S^{k_1+k_2}
$$

---

## 🎵 Practical Summary

- Notes = basis vectors in $ \mathbb{R}^{12} $
- Chords = sums of these vectors
- Transposition = multiply by $S^k$
- Scales = $ \{ S^{k_j} e_{\text{tonic}} \} $
- Modulation = change tonic + apply $S^k$

**Music → Linear algebra**

$$
\boxed{\text{Transpose by } k \text{ semitones} \iff v \mapsto S^k v}
$$


In [16]:
def shift_matrix(k):
    return (np.roll(np.eye(12), k, axis=0))

In [22]:
Cmaj = np.array([1,0,0,0,1,0,0,1,0,0,0,0])
for i in range(0,13,2):
    play_chord(generate_transform(4)@shift_matrix(i)@Cmaj)

In [19]:
print(shift_matrix()@Cmaj)

[0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0.]


# Chord Generation Using the Shift Matrix in $ \mathbb{R}^{12} $

In the pitch–class model, each note is represented as a one–hot vector in $ \mathbb{R}^{12} $.
For example, with the ordering $[C, C#, D, \dots, B]$:

$$
e_0 = (1,0,0,0,0,0,0,0,0,0,0,0)^T, \qquad
e_4 = (0,0,0,0,1,0,0,0,0,0,0,0)^T, \quad \text{etc.}
$$

---

## 1. Semitone Shift Matrix

We define a linear operator $S \in \mathbb{R}^{12 \times 12}$ by:

$$
S e_k = e_{(k+1) \bmod 12} ,
$$

i.e. $S$ shifts each pitch class **up by one semitone**.  
Powers of $S$ produce larger transpositions:

$$
S^k e_j = e_{(j+k) \bmod 12}.
$$

Thus:

- $S^4$ shifts a note up a **major third**
- $S^7$ shifts a note up a **perfect fifth**

---

## 2. Chords as Linear Operators

A chord is generated by adding shifted copies of the root.  
For a chord type with semitone offsets $k_1, k_2, \dots, k_m$, the **chord operator** is:

$$
M = I + S^{k_1} + S^{k_2} + \cdots + S^{k_m}.
$$

Applying this operator to a root vector $e_r$ gives the chord:

$$
v_{\text{chord}} = M \, e_r.
$$

This expresses a chord as a **linear transformation** of a single note.

---

## 3. Example: Major Triad

A major chord consists of:

- root: $0$ semitones
- major third: $4$ semitones
- perfect fifth: $7$ semitones

Hence the **major chord operator** is:

$$
M_{\text{maj}} = I + S^4 + S^7 .
$$

Given a root pitch class $e_r$, the major triad is:

$$
v_{\text{maj}}(r) = M_{\text{maj}} e_r 
= e_r + S^4 e_r + S^7 e_r.
$$

Example (root C = $e_0$):

$$
v_{\text{Cmaj}} = e_0 + S^4 e_0 + S^7 e_0
= e_0 + e_4 + e_7,
$$

which corresponds to the notes: **C–E–G**.

---

## 4. Why This Matters

This formulation makes chord operations purely linear:

- **Transposition:** multiply by $S^k$
- **Chord generation:** apply an operator $M$
- **Modulation:** change root $e_r$ and reuse $M$

It represents harmony using **linear algebra over $ \mathbb{R}^{12} $**, rather than note names or music-specific logic.

---

### 🔑 Final Interpretation

$$
\boxed{
\text{Chord} = (\text{Sum of shifted copies of the root})
= (I + S^{k_1} + \cdots + S^{k_m})\, e_{\text{root}}
}
$$

> **Harmony becomes a matrix operation.**


In [106]:
PITCH_NAMES = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]

# 12×12 Identity → One-hot vectors for notes
vectors = np.eye(12)

# Build DataFrame
df_notes = pd.DataFrame({
    "Note": PITCH_NAMES,
    "Vector": [v for v in vectors]
})

In [63]:
CHORD_STRUCTURES = {
    "major": {
        "triad": [0, 4, 7],        # major triad
        "7":     [0, 4, 7, 10],    # dominant 7
        "maj7":  [0, 4, 7, 11],    # major 7
    },
    "minor": {
        "triad": [0, 3, 7],        # minor triad
        "7":     [0, 3, 7, 10],    # minor 7
    },
    "diminished": {
        "triad": [0, 3, 6],        # dim triad
        "7":     [0, 3, 6, 9],     # fully dim 7
        "m7b5":  [0, 3, 6, 10],    # half-diminished (min7b5)
    },
    "augmented": {
        "triad": [0, 4, 8],        # augmented triad
    },
    "suspended": {
        "sus2":  [0, 2, 7],        # sus2 chord
        "sus4":  [0, 5, 7],        # sus4 chord
    },
    "power": {
        "5":     [0, 7],           # power chord (root + 5th)
    }
}



In [87]:
def get_note_vector(note_name, df=df_notes):
    note = note_name.upper().replace("♯", "#")
    match = df[df["Note"].str.upper() == note]
    if match.empty:
        raise ValueError(f"Note '{note_name}' not found in DataFrame.")
    return match["Vector"].values[0]

def chord_op(chord_type, subtype=None):
    t = chord_type.lower()
    st = subtype.lower() if isinstance(subtype, str) else None
    if st is None:
        if t in ("major", "minor", "diminished", "augmented"):
            st = "triad"
        elif t == "power":
            st = "5"
        else:
            raise ValueError(f"No default subtype for chord type '{chord_type}'")
    if t not in CHORD_STRUCTURES:
        raise ValueError(f"Unknown chord type: '{chord_type}'")
    if st not in CHORD_STRUCTURES[t]:
        raise ValueError(f"Unknown subtype '{subtype}' for chord type '{chord_type}'")
    intervals = CHORD_STRUCTURES[t][st]
    M = np.zeros((12, 12), dtype=float)
    for k in intervals:
        M += shift_matrix(k % 12)
    return M

def play_chord_type(note_name, octave, chord_type, subtype):
    note_vec   = get_note_vector(note_name)              
    chord_vec  = chord_op(chord_type, subtype) @ note_vec
    freq_vec   = generate_transform(octave) @ chord_vec  
    play_chord(freq_vec)

def maj3(note_name, octave=4):
    play_chord_type(note_name, octave, "major", "triad")

def min3(note_name, octave=4):
    play_chord_type(note_name, octave, "minor", "triad")

def dim3(note_name, octave=4):
    play_chord_type(note_name, octave, "diminished", "triad")

def aug3(note_name, octave=4):
    play_chord_type(note_name, octave, "augmented", "triad")

def dom7(note_name, octave=4):
    play_chord_type(note_name, octave, "major", "7")

def maj7(note_name, octave=4):
    play_chord_type(note_name, octave, "major", "maj7")

def min7(note_name, octave=4):
    play_chord_type(note_name, octave, "minor", "7")

def dim7(note_name, octave=4):
    play_chord_type(note_name, octave, "diminished", "7")

def m7b5(note_name, octave=4):
    play_chord_type(note_name, octave, "diminished", "m7b5")

def sus2(note_name, octave=4):
    play_chord_type(note_name, octave, "suspended", "sus2")

def sus4(note_name, octave=4):
    play_chord_type(note_name, octave, "suspended", "sus4")

def pow5(note_name, octave=4):
    play_chord_type(note_name, octave, "power", "5")

In [149]:
maj3("C", 4)     # C major triad
min3("A", 4)     # A minor triad
dom7("G", 4)     # G7
maj7("F#", 4)    # F#maj7
m7b5("B", 4)     # B half-diminished
sus4("D", 4)     # Dsus4
pow5("E", 2)     # E5 power chord

## Progressions

In [128]:
def build_progression(chords):
    cols = []
    for (note, ctype, sub) in chords:
        v_root = get_note_vector(note)
        v_chord = chord_op(ctype, sub) @ v_root
        cols.append(v_chord)
    return np.column_stack(cols)

def play_progression(C, octave, durations):
    T = generate_transform(octave) 
    F = T @ C                       
    for i in range(C.shape[1]):
        play_chord(F[:, i], duration=durations[i])

In [169]:
progression = [
    ("g", "major", "triad"),  # Em
    ("c", "major", "triad"),  # D (major)
    ("D", "major", "triad"),  # Dm
    ("e", "major", "triad"),  # G7
    ("a", "major", "triad")
]

# Build chord matrix
C = build_progression(progression)

# Durations for each chord (seconds)
dur = [2.0]*8

play_progression(C, octave=4, durations=dur)
